In [472]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

df = pd.read_csv('/Users/seanyu_mac/Downloads/[공구] 일정 신청 및 관리 시트 - 공구전체일정 (4).csv')
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_2024-03-12~2025-03-12.xlsx')
auth_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_pb_user_auth_info_export_2025-03-12_103625.csv')
user_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_pb_user_export_2025-03-12_103644.csv')
sales_df = pd.read_csv('/Users/seanyu_mac/Downloads/query_2-2025-03-12_103837.csv')

df.columns = df.loc[1]
df = df.drop([0,1])
con1 = df['진행상황-업체'] == '업체-일정확정'
con2 = df['진행상황-업체'] == '업체-공구종료'
df = df[con1 | con2]
df = df.reset_index()
df.loc[df['시작일'].str.len() <= 8, '시작일'] = '25/' + df['시작일']

dates = []

for i in df['시작일']:
    date = i.split('(')[0].strip()
    year = '20' + date.split('/')[0].strip()
    month = date.split('/')[1].strip()
    day = date.split('/')[2].strip()
    final = year + '-' + month + '-' + day
    dates.append(final)

df['date'] = dates
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= datetime(2025, 2, 17)]

sales_df['time'] = pd.to_datetime(sales_df['reg_dt']) + timedelta(hours=9)
sales_df['minute'] = sales_df['time'].dt.strftime('%H:%M:%S')
sales_df['minute'] = pd.to_datetime(sales_df['minute'])
sales_df['minute'].mean()

Timestamp('2025-03-12 11:35:06.004580096')

In [474]:
print('2월 4주차 부터 매주 예정된 공구건은')
df.groupby(pd.Grouper(key='date', freq='W'))['진행상황-업체'].count()[:10]

2월 4주차 부터 매주 예정된 공구건은


date
2025-02-23    31
2025-03-02    31
2025-03-09    35
2025-03-16    26
2025-03-23    36
2025-03-30    34
2025-04-06    41
2025-04-13    29
2025-04-20    32
2025-04-27    21
Freq: W-SUN, Name: 진행상황-업체, dtype: int64

In [476]:
print('주간 파마브로스 전체 공구 매출')
sales_df.groupby(pd.Grouper(key='time', freq='W'))['order_price'].sum()

주간 파마브로스 전체 공구 매출


time
2025-01-05     374943300
2025-01-12    1559927300
2025-01-19     981915550
2025-01-26     693340500
2025-02-02     309597100
2025-02-09     484714500
2025-02-16     524649200
2025-02-23     369626300
2025-03-02     123291500
2025-03-09      54438250
2025-03-16       3663000
Freq: W-SUN, Name: order_price, dtype: int64

In [478]:
# 공동구매 상품 번호 입력
status = ['배송중', '배송완료', '구매확정', '배송준비', '결제완료']
gongu = [50557]

fs_df = fs_df[fs_df['주문 상태'].isin(status)]
fs_df = fs_df[fs_df['상품 번호'].isin(gongu)]

print('매주 친한스토어 공구 매출은')
fs_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['정산가'].sum()

매주 친한스토어 공구 매출은


결제 완료일
2025-02-23    11570400
2025-03-02     4608000
Freq: W-SUN, Name: 정산가, dtype: int64

In [480]:
print('주간 파마브로스 전체 공구 건수')
sales_df.groupby(pd.Grouper(key='time', freq='W'))['order_no'].count()

주간 파마브로스 전체 공구 건수


time
2025-01-05     3398
2025-01-12    13723
2025-01-19     9185
2025-01-26     6301
2025-02-02     3065
2025-02-09     3428
2025-02-16     5446
2025-02-23     3623
2025-03-02     1321
2025-03-09      464
2025-03-16       46
Freq: W-SUN, Name: order_no, dtype: int64

In [482]:
print('매주 친한스토어 공구 매출 건수는')
fs_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['결제 번호'].size()

매주 친한스토어 공구 매출 건수는


결제 완료일
2025-02-23    71
2025-03-02    28
Freq: W-SUN, Name: 결제 번호, dtype: int64

In [504]:
status = ['배송중', '배송완료', '구매확정', '배송준비', '결제완료']
fs_df = fs_df[fs_df['주문 상태'].isin(status)]

print('매주 목요일 기준 친한스토어 총 매출은')
fs_df.groupby(pd.Grouper(key='결제 완료일', freq='W'))['정산가'].sum()

매주 목요일 기준 친한스토어 총 매출은


결제 완료일
2025-02-23    11570400
2025-03-02     4608000
Freq: W-SUN, Name: 정산가, dtype: int64

In [506]:
merged_df = user_df.merge(auth_df, left_on='id', right_on='user_id', how='inner')
merged_df = user_df.merge(auth_df, left_on='id', right_on='user_id', how='inner')
merged_df['reg_dt'] = pd.to_datetime(merged_df['reg_dt'], format='mixed')
con1 = merged_df['reg_dt'] >= datetime(2025, 2, 17)
con2 = merged_df['reg_dt'] < datetime(2025, 2, 24)
print('매주 친한스토어 친약+친스 회원가입자 수는')
merged_df.groupby(pd.Grouper(key='reg_dt', freq='W')).size()[110:]

매주 친한스토어 친약+친스 회원가입자 수는


reg_dt
2025-01-05    476
2025-01-12    507
2025-01-19    431
2025-01-26    309
2025-02-02    428
2025-02-09    407
2025-02-16    343
2025-02-23    625
2025-03-02    569
2025-03-09    622
2025-03-16    173
Freq: W-SUN, dtype: int64

In [508]:
print('친스 회원가입자 수는 (di 없는 유저만)')
auth_df_di = auth_df[auth_df['di'].isna()]
auth_df_di['created_at'] = pd.to_datetime(auth_df_di['created_at'], format='mixed')
auth_df_di.groupby(pd.Grouper(key='created_at', freq='W')).size()

친스 회원가입자 수는 (di 없는 유저만)


created_at
2025-02-23    123
2025-03-02     26
Freq: W-SUN, dtype: int64